In [ ]:
from fase2_funciones import *
from Apriori import *
import os
import subprocess

df_bd_preprocesada = pd.read_csv('BD/OnlineRetail_Preprocesada.csv', sep=',', encoding = 'unicode_escape')
df_pivot = df_bd_preprocesada['InvoiceDate'].str.split(expand = True)
df_pivot.rename(columns = {0: 'Fecha', 1: 'Hora'}, inplace=True)
df_bd_preprocesada = df_bd_preprocesada.drop(columns =['Unnamed: 0','InvoiceDate'])
df_bd_preprocesada = pd.concat([df_bd_preprocesada,df_pivot], axis = 1)
df_bd_preprocesada['Monto'] = df_bd_preprocesada['UnitPrice'] * df_bd_preprocesada['Quantity']

In [ ]:
#--------------------Llamado a funciones ------------------
articulos_libras_promedio (df_bd_preprocesada)
grupo_cliente(df_bd_preprocesada)
numero_transacciones(df_bd_preprocesada)

In [ ]:
#--------------------Se ejecuta el archivo Apriori.py-------------
a_priori(df_bd_preprocesada)